In [ ]:
import sisl
import hubbard as hubb
import hubbard.sp2 as sp2
import hubbard.density as density
import hubbard.plot as plot
%matplotlib inline

# Simulation of electron correlations in molecules

In this example we will study the electrons correlation in a finite molecule, and find the self-consistent solution using the `hubbard` package.

We will use the molecule of ref. [Nature Communications 10, 200 (2019)](https://www.nature.com/articles/s41467-018-08060-6) and compare the simulation with the experimental system.

You can find the molecular geometry in the file `junction-2-2.XV`.

### Getting started:

We will start by building the tight-binding (TB) Hamiltonian for an sp2 carbon-based molecule, by first reading the geometry file using `sisl`. To build the TB Hamiltonian (which describes the kinetic part of the system's Hamiltonian) we will use the function from the `hubbard` package `sp2` that builds the TB `sisl.Hamiltonian` of an sp2 carbon system. In particular, we will use the set of parameters `t1=2.7`,`t2=0.2` and `t3=0.18` eV to model this Hamiltonian.

In [ ]:
# Build sisl.Geometry object from the 'XV' file
geom = sisl.get_sile('junction-2-2.XV').read_geometry()

# Build sisl.Hamiltonian object using the sp2 function
Hsp2 = sp2(geom, t1=2.7, t2=0.2, t3=0.18)

We will build the `HubbardHamiltonian` object, which will enable the routines to iterate the mean field Hubbard Hamiltonian until it finds the self-consistent solution (convergence). To model the interaction part (Hubbard term) we will use `U=3.5` eV. You can find the parameters used to build the full Hamiltonian (both the kinetic and interaction parts) in the [Supp. Material](https://www.nature.com/articles/s41467-018-08060-6#Sec12) of the [paper referenced above](https://www.nature.com/articles/s41467-018-08060-6) and references therein.


For more information type:

```
help(hubb.HubbardHamiltonian)
```

In [ ]:
# Build the HubbardHamiltonian object with U=3.5 at a temperature of kT~0 in units of the Boltzmann constant
HH = hubb.HubbardHamiltonian(Hsp2, U=3.5, kT=1e-5)

##### Important note before starting the convergence process
It is important to initialize the convergence of the `HubbardHamiltonian` object by giving an initial spin-density that breaks the symmetry between the up- and down- spin channels. Otherwise the code *will not be able to find a solution*.
Furthermore, the closer the initial spin-densities are to the self-consistent solution, the faster the code will find it.

In [ ]:
# Firstly we have to initialize the spin-densities distribution (polarization)
# The polarization can be tuned by using the following function to start with a
# specific up- and down-spin density distribution. In this case we will start
# by placing up-spin components at atomic postion 77 and down-spin components at position 23
HH.set_polarization([77], dn=[23])

Now we can start the convergence until we find the self-consistent solution up to a desired tolerance (`tol`) by calling the `HubbardHamiltonian.converge` method. This method needs as a mandatory argument another function that returns the spin-densities, since depending on the system or the boundary conditions the spin-densities will be obtain differently. For instance, to compute the spin-densities for TB Hamiltonians with *finite or periodic* boundary conditions, one can use the method `hubbard.density.calc_n`.

Type:

```
help(hubb.HubbardHamiltonian.converge)
```
and/or

```
help(density.calc_n)
```

for more information.

In [ ]:
# Converge until a tolerance of tol=1e-10
# With print_info=True it prints (dn,  Etot) for each 10 iterations
# where dn is the spin-densities difference between last and current iteration and Etot is the total energy
dn = HH.converge(density.calc_n, tol=1e-10, print_info=True, steps=10)

##### Understanding the final results
During the iteration process the total energy has also been calculated. Now that the convergence has been achieved the total energy for the self-consistent solution is stored in the `HubbardHamiltonian` object. We can save this value to compare it with further calculations.

Also, we can visualize some meaningful physical quantities and properties of the solution, e.g. such as the spin polarization. Other interesting electronic properties can be visualized using the `hubbard.plot` module (take a look at the exercises section below)

In [ ]:
# Save total energy
E_0 =  HH.Etot
print(f"Total energy = {E_0}")

# Let's visualize some relevant physical quantities of the final result (this process may take a few seconds):
# By passing the argument ext_geom one can plot the spin polarization using the full geometry for the sp2 system
# e.g. including Hydrogen atoms. Otherwise it only displays the carbon backbone (pi-network) structure
p = plot.SpinPolarization(HH, colorbar=True, vmax=0.4, vmin=-0.4, ext_geom=geom)
p.annotate()

##### Using output of the mean-field Hubbard model to start SIESTA calculation
The `hubbard` package can be used to give a starting spin-density distribution for a more accurate spin polarized [SIESTA](https://gitlab.com/siesta-project/siesta) calculation, by writing the spin densities to a `fdf-block`:

In [ ]:
# Write the self-consistent solution of the mean-field Hubbard calculation
# to an input file for a SIESTA calculation.
# ext_geom is the full sp2 geometry that includes the Hydrogen atoms,
# otherwise it uses only the carbon backbone structure (pi-network) stored in the Hsp2 Hamiltonian
HH.write_initspin('init-spin.fdf', ext_geom=geom)

## Try it yourself

#### Simulation of the singlet-triplet transtition energy

For this first example we will continue with the example from above and will use the `hubbard` capabilities to perform further simulations.
In absence of specifications, since the molecule has an even number of total electrons, the charge associated to the up- and down-spin channels is equal (`ntot/2`).
Therefore, we have just found the antiferromagnetic solution. 
But we can manipulate and compute the solution for the ferromagnetic case (or other magnetic states), by imposing an imbalance between up- and down-spin components. 

To find the ferromagnetic solution one just have to do:

```
HH.q[0] += 1
HH.q[1] -= 1
```

and converge again.

You can now plot the spin polarization of the ferromagnetic solution and compare with the previous solution.

Now compare the total energies, this will tell you which solution is the ground state.

##### Extra material:
You can go to the [Supp. Material](https://www.nature.com/articles/s41467-018-08060-6#Sec12) of the [referenced paper above](https://www.nature.com/articles/s41467-018-08060-6), and try to reproduce other results, e.g. such as the singlet triplet transition curve as a function of the Coulomb parameter `U`, the wavefunctions for each spin-channel, etc.